In [10]:
from pyspark.sql import Row
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

In [13]:
conf = SparkConf().setMaster("local").setAppName("Practice")
spark = SparkSession.builder.appName("Practice").getOrCreate()
sc = SparkContext.getOrCreate(conf)

In [14]:
df_dup = sc.parallelize([Row(server_name='101 Server', cpu_utilization = 85, session_count = 80),
                         Row(server_name='101 Server', cpu_utilization = 80, session_count = 90),
                         Row(server_name='102 Server', cpu_utilization = 85, session_count = 80),
                         Row(server_name='102 Server', cpu_utilization = 85, session_count = 80)]).toDF()

In [15]:
df_dup.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           80|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



In [16]:
df_dup.drop_duplicates().show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 101 Server|             80|           90|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



In [17]:
df_dup.drop_duplicates(['server_name']).show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 Server|             85|           80|
| 102 Server|             85|           80|
+-----------+---------------+-------------+



In [18]:
df_na = df_dup.withColumn('na_col', lit(None).cast(StringType()))

In [19]:
df_na.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|  NULL|
| 101 Server|             80|           90|  NULL|
| 102 Server|             85|           80|  NULL|
| 102 Server|             85|           80|  NULL|
+-----------+---------------+-------------+------+



In [20]:
df_na.fillna('A').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           80|     A|
| 102 Server|             85|           80|     A|
+-----------+---------------+-------------+------+



In [21]:
df_2 = df_na.fillna('A').union(df_na)
df_2.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           80|     A|
| 102 Server|             85|           80|     A|
| 101 Server|             85|           80|  NULL|
| 101 Server|             80|           90|  NULL|
| 102 Server|             85|           80|  NULL|
| 102 Server|             85|           80|  NULL|
+-----------+---------------+-------------+------+



In [22]:
df_2.createOrReplaceTempView('na_table')

In [25]:
spark.sql('SELECT * FROM na_table').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           80|     A|
| 102 Server|             85|           80|     A|
| 101 Server|             85|           80|  NULL|
| 101 Server|             80|           90|  NULL|
| 102 Server|             85|           80|  NULL|
| 102 Server|             85|           80|  NULL|
+-----------+---------------+-------------+------+



In [26]:
spark.sql('SELECT * \
          FROM na_table \
          WHERE na_col IS NULL').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|  NULL|
| 101 Server|             80|           90|  NULL|
| 102 Server|             85|           80|  NULL|
| 102 Server|             85|           80|  NULL|
+-----------+---------------+-------------+------+



In [36]:
df_3 = df_2.dropna()
df_3.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 Server|             85|           80|     A|
| 101 Server|             80|           90|     A|
| 102 Server|             85|           80|     A|
| 102 Server|             85|           80|     A|
+-----------+---------------+-------------+------+

